In [1]:
pip install apache_beam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.3/140.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.6/603.6 kB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 60.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 26.0 MB/s eta 0:00:00
  Created wheel for crcmod: filename=crcmod-1.7-cp310-cp310-linux_x86_64.whl size=31404 sha256=56d9eceaba6fe381b7

APACHE BEAM TRANSFORM CATALOG ELEMENT-WISE ORNEKLERİ VE KULLANIMLARI

# FİLTER

Bu fonksiyon geneler olarak apache-beam ile çalışırken veriler içinde belirli bir ayrıma göre verilerinizi seçmek sınıflandırmak için kullanabileceğiniz bir yöntemdir. Oluşturduğunuz Data Pipeline içinde bu verileri filtreleyerek bir sonraki koloksiyonunuzda kullanabileceğiniz bir hale getirebilirsiniz. alternetif kullanımları aşağıda gösterilmiştir. referans: https://beam.apache.org/documentation/transforms/python/elementwise/filter/

In [2]:
import apache_beam as beam

In [3]:
"""
Bu kullanımda bir sözlük yapısında olan verilerimiz ile çalışıyoruz. Her zaman szölük yapısına uygun veriler ile çalışmayabiliriz ama filtreleme işlemleri için her zaman
sözlük yapısna uygun şekilde dönüştürülmüş veriler kolaylık sağlamaktadır. responsu evet olan verileri filtreliyoruz.
"""
def is_yes(data):
  return data['response'] == 'evet' # responseları sadece evet olanları döndürüyor

with beam.Pipeline() as pipeline:
  evet_collection = ( #tanımlanan datalarda responsları evet olanları evet_collection collection olarak atayan pipeline oluşturuyoruz.
      pipeline
      | 'OrnekData' >> beam.Create([
          {
                'name': 'Hakan', 'response': 'evet'
            },
            {
               'name': 'Mehmet', 'response': 'hayır'
            },
            {
                'name': 'Ayşe', 'response': 'hayır'
            },
            {
                'name': 'Fatma', 'response': 'evet'
            },
            {
                 'name': 'Faruk', 'response': 'evet'
            },
      ])
      | 'Filter Data' >> beam.Filter(is_yes)
      | beam.Map(print)
  )

{'name': 'Hakan', 'response': 'evet'}
{'name': 'Fatma', 'response': 'evet'}
{'name': 'Faruk', 'response': 'evet'}


In [4]:
#YUKARIDA YAPMIŞ OLDUĞUMUZ İŞLEMİ LAMBDA FONKSİYONU OLARAK OLUŞTURALIM.

with beam.Pipeline() as pipeline:
  evet_collection = ( #tanımlanan datalarda responsları evet olanları evet_collection collection olarak atayan pipeline oluşturuyoruz.
      pipeline
      | 'OrnekData' >> beam.Create([
          {
                'name': 'Hakan', 'response': 'evet'
            },
            {
               'name': 'Mehmet', 'response': 'hayır'
            },
            {
                'name': 'Ayşe', 'response': 'hayır'
            },
            {
                'name': 'Fatma', 'response': 'evet'
            },
            {
                 'name': 'Faruk', 'response': 'evet'
            },
      ])
      | 'Filter Data' >> beam.Filter(lambda data : data['response'] == 'evet' )
      | beam.Map(print)
  )

{'name': 'Hakan', 'response': 'evet'}
{'name': 'Fatma', 'response': 'evet'}
{'name': 'Faruk', 'response': 'evet'}


In [5]:
"""
Yukarda kullanılan yöntemlerden farklı olarak aşağıdaki iki örnekte Data Pipeline içindeki farklı koleksiyondaki değere göre bi filtreleme işlemi yapıyoruz.
"""
with beam.Pipeline() as pipeline:
  referans_collection = pipeline | 'evet' >> beam.Create(['evet'])

  data = (
      pipeline
      | 'OrnekData' >> beam.Create([
          {
                'name': 'Hakan', 'response': 'evet'
            },
            {
               'name': 'Mehmet', 'response': 'hayır'
            },
            {
                'name': 'Ayşe', 'response': 'hayır'
            },
            {
                'name': 'Fatma', 'response': 'evet'
            },
            {
                 'name': 'Faruk', 'response': 'evet'
            },
      ])
      | 'Filter evet' >> beam.Filter(
          lambda data,
          response : data['response'] == response, #datamız içindeki response değerini yukarıda tanımladığımız collection içerisinden alarak işlemimizi gerçekleştiriyoruz
          response = beam.pvalue.AsSingleton(referans_collection),
      )
      | beam.Map(print)
  )

{'name': 'Hakan', 'response': 'evet'}
{'name': 'Fatma', 'response': 'evet'}
{'name': 'Faruk', 'response': 'evet'}


In [6]:
with beam.Pipeline() as pipeline:
  iterable_value = pipeline | 'İterable Value Create' >> beam.Create([
      'evet',
      'belki',
  ])
  data = (
      pipeline
      | 'OrnekData' >> beam.Create([
          {
                'name': 'Hakan', 'response': 'evet'
            },
            {
               'name': 'Mehmet', 'response': 'hayır'
            },
            {
                'name': 'Ayşe', 'response': 'hayır'
            },
            {
                'name': 'Fatma', 'response': 'evet'
            },
            {
                 'name': 'Faruk', 'response': 'evet'
            },
            {
                 'name': 'Faruk', 'response': 'belki'
            },
      ])
      | 'Filter evet' >> beam.Filter(
          lambda data,
          response : data['response'] in response , #datamız içindeki response değerini yukarıda tanımladığımız collection (birden fazla iterable seçenekler için) içerisinden alarak işlemimizi gerçekleştiriyoruz
          response = beam.pvalue.AsIter(iterable_value),
      )
      | beam.Map(print)
  )

{'name': 'Hakan', 'response': 'evet'}
{'name': 'Fatma', 'response': 'evet'}
{'name': 'Faruk', 'response': 'evet'}
{'name': 'Faruk', 'response': 'belki'}


In [7]:
with beam.Pipeline() as pipeline:
  keep_value = pipeline | 'keep Value Create' >> beam.Create([
      ('evet',False),
      ('belki',True),
      ('hayır',True),
  ])
  data = (
      pipeline
      | 'OrnekData' >> beam.Create([
          {
                'name': 'Hakan', 'response': 'evet'
            },
            {
               'name': 'Mehmet', 'response': 'hayır'
            },
            {
                'name': 'Ayşe', 'response': 'hayır'
            },
            {
                'name': 'Fatma', 'response': 'evet'
            },
            {
                 'name': 'Faruk', 'response': 'evet'
            },
            {
                 'name': 'Faruk', 'response': 'belki'
            },
      ])
      | 'Filter evet' >> beam.Filter(
          lambda data,
          response : response[data['response']] ,
          response = beam.pvalue.AsDict(keep_value),
      )
      | beam.Map(print)
  )

{'name': 'Mehmet', 'response': 'hayır'}
{'name': 'Ayşe', 'response': 'hayır'}
{'name': 'Faruk', 'response': 'belki'}


# FlatMap Fonksiyonu ve Örnek Kullanımları
FlatMap, Apache Beam içindeki bir dönüşüm işlemidir ve veri akışındaki her öğe için bir veya daha fazla çıktı üretebilmenizi sağlar. Her girdi öğesi için birden çok çıktı öğesi üretmek için kullanılırken, Map işlemi yalnızca her girdi öğesi için bir çıktı üretir.

FlatMap işlemi, özellikle bir girdi öğesinden birden fazla çıktı öğesi üretmeniz gereken durumlarda faydalıdır. Örneğin, bir metin içindeki kelimeleri ayırarak her kelimeyi ayrı bir öğe olarak çıkartmak veya bir öğe içindeki alt öğeleri çıkartarak her alt öğeyi ayrı bir çıktı olarak üretmek gibi senaryolarda kullanışlıdır.

In [8]:
"""
Tanımlanan her bir datayı ',' le göre ayırarak her biri üstünde ayrı ayrı işlem yapmak için kullanılabilir
"""
with beam.Pipeline() as pipeline:
  data = (
      pipeline
      | 'Create' >> beam.Create([
          'ahmet','mehmet','mustafa','ali','veli',
      ])
      | 'Split Words' >> beam.FlatMap(str.split)
      | beam.Map(print)
  )

ahmet
mehmet
mustafa
ali
veli


In [31]:
def split_words(text):
  return text.split(',')

with beam.Pipeline() as pipeline:
  data = (
      pipeline
      | 'Create' >> beam.Create([
          'ahmet','mehmet','mustafa','ali','veli',
      ])
      | 'Split Words' >> beam.FlatMap(split_words)
      | beam.Map(print)
  )

ahmet
mehmet
mustafa
ali
veli


In [29]:
"""
Liste şeklinde tanımlanan veriyi ayırmak için kullanılabilir
"""
with beam.Pipeline() as pipeline:
  data = (
      pipeline
      | 'Create' >> beam.Create([
          ['ahmet','mehmet','mustafa','ali','veli'],
          ['ahmet','mehmet','mustafa','ali','veli'],
      ])
      | 'Flatten list' >> beam.FlatMap(lambda elements : elements)
      | beam.Map(print)
  )

ahmet
mehmet
mustafa
ali
veli
ahmet
mehmet
mustafa
ali
veli


In [11]:
def generate_elements(elements):
  for element in elements:
    yield element

with beam.Pipeline() as pipeline:
  data = (
      pipeline
      | 'Create' >> beam.Create([
          ['ahmet','mehmet','mustafa'],
           ['ali','veli'],
      ])
      | 'Flatten list' >> beam.FlatMap(generate_elements)
      | beam.Map(print)
  )

ahmet
mehmet
mustafa
ali
veli


In [12]:
def format_data(name,surname):
  if name:
    yield '{}{}'.format(name,surname)

with beam.Pipeline() as pipeline:
  data = (
      pipeline
      | 'Create' >> beam.Create([
          ('ahmet','sargın'),
          ('ahmet','faruk'),
          ('ahmet','ahmet'),
      ])
      | 'Flatten list' >> beam.FlatMapTuple(format_data)
      | beam.Map(print)
  )

ahmetsargın
ahmetfaruk
ahmetahmet


In [13]:
"""
Pipeline içinde tanımlanmış olan farklı bir koleksiyon içindeki veriyi kullanmak için.
"""

with beam.Pipeline() as pipeline:
    delimiter = pipeline | 'Create delimiter' >> beam.Create([','])

    plants = (
        pipeline
        | 'Gardening plants' >> beam.Create([
            '🍓Strawberry,🥕Carrot,🍆Eggplant',
            '🍅Tomato,🥔Potato',
        ])
        | 'Split words' >> beam.FlatMap(
            lambda text,
            delimiter: text.split(delimiter),
            delimiter=beam.pvalue.AsSingleton(delimiter),
        )
        | beam.Map(print))
    # [END flatmap_side_inputs_singleton]


🍓Strawberry
🥕Carrot
🍆Eggplant
🍅Tomato
🥔Potato


In [14]:
def normalize_and_validate_durations(plant, valid_durations):
    plant['duration'] = plant['duration'].lower()
    if plant['duration'] in valid_durations:
      yield plant

with beam.Pipeline() as pipeline:
    valid_durations = pipeline | 'Valid durations' >> beam.Create([
        'annual',
        'biennial',
        'perennial',
    ])

    valid_plants = (
        pipeline
        | 'Gardening plants' >> beam.Create([
            {
                'icon': '🍓', 'name': 'Strawberry', 'duration': 'Perennial'
            },
            {
                'icon': '🥕', 'name': 'Carrot', 'duration': 'BIENNIAL'
            },
            {
                'icon': '🍅', 'name': 'Tomato', 'duration': 'annual'
            },
            {
                'icon': '🥔', 'name': 'Potato', 'duration': 'unknown'
            },
        ])
        | 'Normalize and validate durations' >> beam.FlatMap(
            normalize_and_validate_durations,
            valid_durations=beam.pvalue.AsIter(valid_durations),
        )
        | beam.Map(print))


{'icon': '🍓', 'name': 'Strawberry', 'duration': 'perennial'}
{'icon': '🥕', 'name': 'Carrot', 'duration': 'biennial'}
{'icon': '🍅', 'name': 'Tomato', 'duration': 'annual'}


# MAP FONKSİYONU KULLANIMI

Map, Apache Beam içindeki bir dönüşüm işlemidir ve veri akışındaki her öğe için bir çıktı öğesi üretebilmenizi sağlar. Map işlemi, veri akışındaki her öğeyi dönüştürmek veya işlemek için kullanılır. Her girdi öğesi için yalnızca bir çıktı öğesi üretir.

Map işlemi, özellikle her girdi öğesini tek bir çıktı öğesiyle değiştirmeniz veya dönüştürmeniz gereken durumlarda kullanışlıdır. Örneğin, veri akışındaki sayıları iki katına çıkartmak veya metinleri büyük harflere dönüştürmek gibi işlemlerde kullanılabilir.

In [15]:
def strip_header_and_newline(text):
  return text.strip('\n')

with beam.Pipeline() as pipeline:
  name = (
      pipeline
      | 'Name' >> beam.Create([
          'ahmet\n',
          'faruk\n',
          'sargın\n',
      ])
      | beam.Map(print)
  )

ahmet

faruk

sargın



# REGEX

Apache Beam içinde düzenli ifadeler (regex) kullanarak veriyi işlemek için beam.Regex modülünü kullanabilirsiniz. Bu modül, düzenli ifadeleri uygulamak ve veri akışındaki metinleri bu ifadelere göre işlemek için kullanılır.

In [33]:
"""
Bu örnekte, regex_pattern adlı bir düzenli ifade tanımlanmış. Bu ifade, ABD'deki sosyal güvenlik numaralarını (SSN) temsil eden tipik bir formata uyar.
Veri akışı içindeki metinler, bu düzenli ifade ile karşılaştırılır ve eşleşen metinler sonuç olarak elde edilir.

beam.Regex.matches() işlemi, düzenli ifadeye uyan metinleri bir PCollection olarak döndürür. Eşleşen metinler bu PCollection içinde yer alır.
Bu örnekte, eşleşen SSN'leri bulmak için düzenli ifade kullanıldı. Gerçek senaryolarda, farklı düzenli ifadeler kullanarak metin analizi, veri temizleme veya veri işleme işlemleri yapabilirsiniz.
"""
regex_pattern = r'\d{3}-\d{2}-\d{4}'

with beam.Pipeline() as pipeline:
    data = pipeline | 'Create input' >> beam.Create([
        '123-45-6789',
        '987-65-4321',
        'invalid-format',
        '555-666-7777'
    ])

    matched_data = data | 'Match regex' >> beam.Regex.matches(regex_pattern)

    matched_data | beam.Map(print)

123-45-6789
987-65-4321


In [20]:
# Matches a named group 'icon', and then two comma-separated groups.
regex = r'(?P<icon>[^\s,]+), *(\w+), *(\w+)'
with beam.Pipeline() as pipeline:
    plants_matches = (
        pipeline
        | 'Garden plants' >> beam.Create([
            '🍓, Strawberry, perennial',
            '🥕, Carrot, biennial ignoring trailing words',
            '🍆, Eggplant, perennial',
            '🍅, Tomato, annual',
            '🥔, Potato, perennial',
            '# 🍌, invalid, format',
            'invalid, 🍉, format',
        ])
        | 'Parse plants' >> beam.Regex.matches(regex)
        | beam.Map(print))
"""
Bu kod parçası, Apache Beam kullanarak veriyi bir düzenli ifade (regex) kullanarak eşleştirme işlemi gerçekleştirerek,
regex = r'(?P<icon>[^\s,]+), *(\w+), *(\w+)': Bu satırda, regex adlı bir düzenli ifade (regex) tanımlanır. Bu düzenli ifade, belirli bir kalıba uyan metinleri eşleştirmek için kullanılacaktır.
 Kalıp, bir emoji (icon), ardından iki virgülle ayrılmış kelime gruplarını içerir.
"""

🍓, Strawberry, perennial
🥕, Carrot, biennial
🍆, Eggplant, perennial
🍅, Tomato, annual
🥔, Potato, perennial


"\nBu kod parçası, Apache Beam kullanarak veriyi bir düzenli ifade (regex) kullanarak eşleştirme işlemi gerçekleştirerek,\nregex = r'(?P<icon>[^\\s,]+), *(\\w+), *(\\w+)': Bu satırda, regex adlı bir düzenli ifade (regex) tanımlanır. Bu düzenli ifade, belirli bir kalıba uyan metinleri eşleştirmek için kullanılacaktır.\n Kalıp, bir emoji (icon), ardından iki virgülle ayrılmış kelime gruplarını içerir.\n"

In [18]:
regex = r'(?P<icon>[^\s,]+), *(\w+), *(\w+)'
with beam.Pipeline() as pipeline:
    plants_all_matches = (
        pipeline
        | 'Garden plants' >> beam.Create([
            '🍓, Strawberry, perennial',
            '🥕, Carrot, biennial ignoring trailing words',
            '🍆, Eggplant, perennial',
            '🍅, Tomato, annual',
            '🥔, Potato, perennial',
            '# 🍌, invalid, format',
            'invalid, 🍉, format',
        ])
        | 'Parse plants' >> beam.Regex.all_matches(regex)
        | beam.Map(print))

['🍓, Strawberry, perennial', '🍓', 'Strawberry', 'perennial']
['🥕, Carrot, biennial', '🥕', 'Carrot', 'biennial']
['🍆, Eggplant, perennial', '🍆', 'Eggplant', 'perennial']
['🍅, Tomato, annual', '🍅', 'Tomato', 'annual']
['🥔, Potato, perennial', '🥔', 'Potato', 'perennial']


In [21]:
with beam.Pipeline() as pipeline:
    plants_replace_all = (
        pipeline
        | 'Garden plants' >> beam.Create([
            '🍓 : Strawberry : perennial',
            '🥕 : Carrot : biennial',
            '🍆\t:\tEggplant\t:\tperennial',
            '🍅 : Tomato : annual',
            '🥔 : Potato : perennial',
        ])
        | 'To CSV' >> beam.Regex.replace_all(r'\s*:\s*', ',')
        | beam.Map(print))

🍓,Strawberry,perennial
🥕,Carrot,biennial
🍆,Eggplant,perennial
🍅,Tomato,annual
🥔,Potato,perennial


In [22]:
with beam.Pipeline() as pipeline:
    plants_replace_first = (
        pipeline
        | 'Garden plants' >> beam.Create([
            '🍓, Strawberry, perennial',
            '🥕, Carrot, biennial',
            '🍆,\tEggplant, perennial',
            '🍅, Tomato, annual',
            '🥔, Potato, perennial',
        ])
        | 'As dictionary' >> beam.Regex.replace_first(r'\s*,\s*', ': ')
        | beam.Map(print))


🍓: Strawberry, perennial
🥕: Carrot, biennial
🍆: Eggplant, perennial
🍅: Tomato, annual
🥔: Potato, perennial


In [23]:
with beam.Pipeline() as pipeline:
    plants_split = (
        pipeline
        | 'Garden plants' >> beam.Create([
            '🍓 : Strawberry : perennial',
            '🥕 : Carrot : biennial',
            '🍆\t:\tEggplant : perennial',
            '🍅 : Tomato : annual',
            '🥔 : Potato : perennial',
        ])
        | 'Parse plants' >> beam.Regex.split(r'\s*:\s*')
        | beam.Map(print))

['🍓', 'Strawberry', 'perennial']
['🥕', 'Carrot', 'biennial']
['🍆', 'Eggplant', 'perennial']
['🍅', 'Tomato', 'annual']
['🥔', 'Potato', 'perennial']


# ParDo

ParDo, Apache Beam içindeki temel bir dönüşüm işlemidir. "Paralel Do"nun kısaltmasıdır ve veri akışındaki her öğe üzerinde özel bir işlem yapmanızı sağlar. Map işlemi gibi, her girdi öğesi için bir çıktı öğesi üretir. Ancak ParDo, daha karmaşık işlemler yapmak ve yan etkiler (side effects) oluşturmak için daha esnek bir seçenektir.

ParDo işlemi, bir fonksiyonu (ParDo işlevi veya ParDo fonksiyonu) her girdi öğesi için çağırarak çalışır. Bu fonksiyon, veri akışındaki öğeleri işleyebilir, yeni öğeler üretebilir ve yan etkiler oluşturabilir.

In [32]:
class ToUpper(beam.DoFn):
    def process(self, element):
        return [element.upper()]

with beam.Pipeline() as pipeline:
    input_data = pipeline | 'Create input' >> beam.Create(['hello', 'world', 'apache', 'beam'])

    uppercased_data = input_data | 'Convert to uppercase' >> beam.ParDo(ToUpper())

    uppercased_data | beam.Map(print)

HELLO
WORLD
APACHE
BEAM


In [40]:
import apache_beam as beam
from collections import Counter

class SplitWords(beam.DoFn):
    def __init__(self, delimiter=','): #Dataları neye göre ayıracağımızı belirtiyoruz
      self.delimiter = delimiter

    def process(self, text):
      for word in text.split(self.delimiter):
        yield word  """Bir fonksiyon içinde yield ifadesi kullanıldığında, fonksiyon artık bir jeneratör olarak kabul edilir. yield ifadesi,
                        fonksiyonun bir değeri ürettiği ve geçici olarak çalışmayı duraklattığı bir noktadır.
                          Jeneratör bu duraklamayı hatırlar ve bir sonraki çağrıda nerede kaldığını bilir."""

with beam.Pipeline() as pipeline:
    plants = (
        pipeline
        | 'Gardening plants' >> beam.Create([
            '🍓Strawberry,🥕Carrot,🍆Eggplant',
            '🍅Tomato,🥔Potato',
        ])
        | 'Split words' >> beam.ParDo(SplitWords(','))
        | beam.Map(print))

🍓Strawberry
🥕Carrot
🍆Eggplant
🍅Tomato
🥔Potato
